In [99]:
#import the necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json 
%matplotlib inline

In [100]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [101]:
#Get the table from Wikipedia
toronto_html_table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)
toronto_df = toronto_html_table[0]
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [102]:
toronto_df.shape

(289, 3)

In [103]:
# Checking for null values
print(toronto_df.isnull().sum())

Postcode         0
Borough          0
Neighbourhood    0
dtype: int64


In [117]:
#Rename the columns
toronto_df.rename(columns = {"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)

#replace ""Not assigned" to Nan
toronto_df.replace("Not assigned", np.nan, inplace=True)

#simply drop whole row with NaN in "Borough" column
toronto_df.dropna(subset=["Borough"],axis=0, inplace=True)

#reset index because we dropped row
toronto_df.reset_index(drop=True,inplace=True)

#Combine the rows with the same "Postcode", and make sure the neighbourhoods are separated with a comma
#toronto_df = toronto_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
toronto_df = toronto_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ", ".join(x.astype(str))).reset_index()
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [118]:
#Make sure for any cell that has a borough and a "nan" neighbourhood, then the neighbourhood will be the same as borough
for borough_item,neighbourhood_item in zip(toronto_df.Borough, toronto_df.Neighborhood): 
    if neighbourhood_item == "nan":
        toronto_df.Neighborhood.replace(['nan'], [borough_item], inplace=True)
               
toronto_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [119]:
#print the number of rows in the dataframe
toronto_df.shape

(103, 3)

In [120]:
#Getting the latitude and longitude for the Postal Code
path="http://cocl.us/Geospatial_data"
lat_long = pd.read_csv(path)

#Rename column to have it identical with the similar column from postalCodes
lat_long.rename(columns={"Postal Code":"PostalCode"},inplace=True)
lat_long.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [121]:
lat_long.shape

(103, 3)

In [122]:
#Merge the two dataframes on PostCode as a key
toronto_geo = pd.merge(toronto_df, lat_long, on='PostalCode')
toronto_geo

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [123]:
#Check how many boroughs and neighbourhoods 
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(toronto_geo['Borough'].unique()),
        toronto_geo.shape[0]
    )
)

Toronto has 11 boroughs and 103 neighborhoods.


In [124]:
toronto_geo_code = toronto_geo['Borough'].astype('category').cat.codes #assign codes to each borough
toronto_data = pd.concat([toronto_geo, toronto_geo_code], axis=1) #merge the codes with the data
toronto_data.columns=['PostalCode','Borough','Neighborhood','Latitude','Longitude','Code'] #rename columns
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Code
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,8
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,8
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,8
3,M1G,Scarborough,Woburn,43.770992,-79.216917,8
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,8


In [125]:
toronto_data.dtypes

PostalCode       object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
Code               int8
dtype: object

In [126]:
# Find out the coordinate of Toronto
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(toronto_latitude, toronto_longitude))

C:\Users\IBM_ADMIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [127]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

neighborhoods = 11

# set color scheme for the clusters
x = np.arange(neighborhoods)
ys = [i+x+(i*x)**2 for i in range(neighborhoods)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, borough, neighborhood, code in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood'], toronto_data['Code']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[code-1],
        fill=True,
        fill_color=rainbow[code-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [128]:
# We simplify the above map and segment and cluster only with boroughs that contain the word Toronto
toronto_boroughs = toronto_data[toronto_data['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_boroughs.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Code
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0


In [129]:
toronto_boroughs.shape

(38, 6)

In [130]:
#Check how many boroughs and neighbourhoods 
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(toronto_boroughs['Borough'].unique()),
        toronto_boroughs.shape[0]
    )
)

Toronto has 4 boroughs and 38 neighborhoods.


In [131]:
# create map of Toronto using latitude and longitude values
toronto_map_boroughs = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=12)

toronto_neigh = 9

# set color scheme for the clusters
x = np.arange(toronto_neigh)
ys = [i+x+(i*x)**2 for i in range(toronto_neigh)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, label, code in zip(toronto_boroughs['Latitude'], toronto_boroughs['Longitude'], toronto_boroughs['Neighborhood'], toronto_boroughs['Code']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[code-1],
        fill=True,
        fill_color=rainbow[code-1],
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map_boroughs)  
    
toronto_map_boroughs

In [132]:
#Explore Toronto neighborhood with Foursquare API

#Define Foursquare credentials and version
CLIENT_ID = 'YSMOX4XNDJEA0KA2U3POQOWDR2W1H1IAZ132F1GJVETNODMY' # your Foursquare ID
CLIENT_SECRET = 'PXQHDCJ4B12ZGXMTUVH1YJIS1XGHZSU3MU5YNJS3OUUSQUDZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YSMOX4XNDJEA0KA2U3POQOWDR2W1H1IAZ132F1GJVETNODMY
CLIENT_SECRET:PXQHDCJ4B12ZGXMTUVH1YJIS1XGHZSU3MU5YNJS3OUUSQUDZ


In [133]:
#Function to explore neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)



In [135]:
toronto_venues = getNearbyVenues(names=toronto_boroughs['Neighborhood'],
                                   latitudes=toronto_boroughs['Latitude'],
                                   longitudes=toronto_boroughs['Longitude']
                                  )

Found 1696 venues in 38 neighborhoods.


In [136]:
print(toronto_venues.shape)
toronto_venues.head()

(1696, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [137]:
#How many venues are returned for each Neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",18,18,18,18,18,18
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [138]:
# Find out how many unique categories from all returned venues
print('There are {} distinct venues in {} categories.'.format(
    len(toronto_venues['Venue'].unique()),len(toronto_venues['Venue Category'].unique())))

There are 1100 distinct venues in 237 categories.


In [139]:
#Analyze each Neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0

In [140]:
#Examine the new dataframe size
toronto_onehot.shape

(1696, 237)

In [141]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.010000,0.010000,0.000000,0.030000,0.0000,0.0000,0.000000,0.0000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.00,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,0.000,0.000000,0.000000,0.000000,0.000000

In [142]:
#Examine the new dataframe size
toronto_grouped.shape

(38, 237)

In [143]:
# Print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.05
2    Cocktail Bar  0.05
3          Bakery  0.04
4  Farmers Market  0.04


----Brockton, Exhibition Place, Parkdale Village----
               venue  freq
0               Café  0.11
1     Breakfast Spot  0.11
2        Coffee Shop  0.11
3  Convenience Store  0.06
4                Gym  0.06


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1       Burrito Place  0.06
2       Auto Workshop  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport S

In [144]:
#Put that into a pandas dataframe

#Write a function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [145]:
#Create a new dataframe and display the top ten venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
toronto_neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Bar,Restaurant,Asian Restaurant,Hotel,Bakery
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Café,Bakery,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market,Pub
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Convenience Store,Performing Arts Venue,Burrito Place,Stadium,Bar,Caribbean Restaurant,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Park,Comic Shop,Recording Studio
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Plane,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Harbor / Marina
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Bakery,Italian Restaurant,Café,Pub,Pizza Place,Breakfast Spot,Sandwich Place
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Chinese Restaurant,Thai Restaurant,Salad Place,Bubble Tea Shop,Spa
7,"Chinatown, Grange Park, Kensington Market",Bar,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Coffee Shop,Bakery,Mexican Restaurant,Chinese Restaurant,Grocery Store
8,Christie,Café,Grocery Store,Park,Italian Restaurant,Athletics & Sports,Restaurant,Diner,Nightclub,Baby Store,Coffee Shop
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Mediterranean Restaurant,Café,Men's Store,Pub


In [146]:
toronto_neighborhoods_venues_sorted.iloc[1,]

Neighborhood                     Berczy Park
1st Most Common Venue            Coffee Shop
2nd Most Common Venue             Restaurant
3rd Most Common Venue           Cocktail Bar
4th Most Common Venue                   Café
5th Most Common Venue                 Bakery
6th Most Common Venue             Steakhouse
7th Most Common Venue               Beer Bar
8th Most Common Venue     Seafood Restaurant
9th Most Common Venue         Farmers Market
10th Most Common Venue                   Pub
Name: 1, dtype: object

In [147]:
# Cluster the Neighborhoods using k-means

#Run K-means to cluster neighborhood into 3 clusters

# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [148]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = toronto_grouped_clustering.columns
means_df.index = ['G1','G2','G3']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Total Sum
G2,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [149]:
#Create a new dataframe that includes the cluster and the top ten venues for each neighborhood

toronto_merged = toronto_boroughs

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,0,Coffee Shop,Pub,Asian Restaurant,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2,0,Park,Ice Cream Shop,Pet Store,Pizza Place,Movie Theater,Pub,Burrito Place,Burger Joint,Sandwich Place,Brewery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Chinese Restaurant,Fish Market,Latin American Restaurant,Brewery
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Women's Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,0,Breakfast Spot,Food & Drink Shop,Sandwich Place,Gym,Burger Joint,Park,Hotel,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,0,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Bagel Shop,Chinese Restaurant,Dessert Shop,Diner,Rental Car Location,Salon / Barbershop
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,0,Sandwich Place,Pizza Place,Dessert Shop,Thai Restaurant,Coffee Shop,Sushi Restaurant,Italian Restaurant,Café,Seafood Restaurant,Burger Joint
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,0,Playground,Gym,Tennis Court,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Ethiopian Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0,0,Coffee Shop,Pub,Pizza Place,American Restaurant,Convenience Store,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint


In [150]:
# create map
toronto_map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_map_clusters)
       
toronto_map_clusters

In [151]:
# Examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

toronto_cluster_0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

toronto_cluster_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

toronto_cluster_2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


In [152]:
toronto_cluster_0

,Borough,Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,2,0,Coffee Shop,Pub,Asian Restaurant,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,East Toronto,2,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
2,East Toronto,2,0,Park,Ice Cream Shop,Pet Store,Pizza Place,Movie Theater,Pub,Burrito Place,Burger Joint,Sandwich Place,Brewery
3,East Toronto,2,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Chinese Restaurant,Fish Market,Latin American Restaurant,Brewery
4,Central Toronto,0,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Women's Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,Central Toronto,0,0,Breakfast Spot,Food & Drink Shop,Sandwich Place,Gym,Burger Joint,Park,Hotel,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
6,Central Toronto,0,0,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Bagel Shop,Chinese Restaurant,Dessert Shop,Diner,Rental Car Location,Salon / Barbershop
7,Central Toronto,0,0,Sandwich Place,Pizza Place,Dessert Shop,Thai Restaurant,Coffee Shop,Sushi Restaurant,Italian Restaurant,Café,Seafood Restaurant,Burger Joint
8,Central Toronto,0,0,Playground,Gym,Tennis Court,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Ethiopian Restaurant
9,Central Toronto,0,0,Coffee Shop,Pub,Pizza Place,American Restaurant,Convenience Store,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint


In [153]:
toronto_cluster_1

,Borough,Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Downtown Toronto,1,1,Coffee Shop,Restaurant,Café,Italian Restaurant,Seafood Restaurant,Pub,Cocktail Bar,Hotel,Beer Bar,Japanese Restaurant


In [155]:
toronto_cluster_2

,Borough,Code,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,1,2,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Chinese Restaurant,Thai Restaurant,Salad Place,Bubble Tea Shop,Spa
22,Central Toronto,0,2,Music Venue,Garden,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
27,Downtown Toronto,1,2,Airport Lounge,Airport Terminal,Airport Service,Plane,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Harbor / Marina


# Explore New York City Neighborhoods

In [156]:
# Accessing the dataset located at: https://geo.nyu.edu/catalog/nyu_2451_34572

#!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
#print('Data downloaded!')

import urllib.request, json 
with urllib.request.urlopen("https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json") as url:
    newyork_data = json.loads(url.read().decode())
    print(newyork_data )
    

{'type': 'FeatureCollection', 'totalFeatures': 306, 'features': [{'type': 'Feature', 'id': 'nyu_2451_34572.1', 'geometry': {'type': 'Point', 'coordinates': [-73.84720052054902, 40.89470517661]}, 'geometry_name': 'geom', 'properties': {'name': 'Wakefield', 'stacked': 1, 'annoline1': 'Wakefield', 'annoline2': None, 'annoline3': None, 'annoangle': 0.0, 'borough': 'Bronx', 'bbox': [-73.84720052054902, 40.89470517661, -73.84720052054902, 40.89470517661]}}, {'type': 'Feature', 'id': 'nyu_2451_34572.2', 'geometry': {'type': 'Point', 'coordinates': [-73.82993910812398, 40.87429419303012]}, 'geometry_name': 'geom', 'properties': {'name': 'Co-op City', 'stacked': 2, 'annoline1': 'Co-op', 'annoline2': 'City', 'annoline3': None, 'annoangle': 0.0, 'borough': 'Bronx', 'bbox': [-73.82993910812398, 40.87429419303012, -73.82993910812398, 40.87429419303012]}}, {'type': 'Feature', 'id': 'nyu_2451_34572.3', 'geometry': {'type': 'Point', 'coordinates': [-73.82780644716412, 40.887555677350775]}, 'geometry_n

In [157]:
# The relevant data is in the features key (n.b. a list of the neighborhoods)
ny_neighborhoods_data = newyork_data['features']

In [158]:
# Check the first item in this list
ny_neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [159]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
ny_neighborhoods = pd.DataFrame(columns=column_names)

In [160]:
for data in ny_neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_neighborhoods = ny_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [161]:
ny_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [162]:
ny_neighborhoods.shape

(306, 4)

In [163]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(ny_neighborhoods['Borough'].unique()),
        ny_neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [164]:
newyork_geo_code = ny_neighborhoods['Borough'].astype('category').cat.codes #assign codes to each borough
newyork_data = pd.concat([ny_neighborhoods, newyork_geo_code], axis=1) #merge the codes with the data
newyork_data.columns=['Borough','Neighborhood','Latitude','Longitude','Code'] #rename columns
newyork_data

,Borough,Neighborhood,Latitude,Longitude,Code
0,Bronx,Wakefield,40.894705,-73.847201,0
1,Bronx,Co-op City,40.874294,-73.829939,0
2,Bronx,Eastchester,40.887556,-73.827806,0
3,Bronx,Fieldston,40.895437,-73.905643,0
4,Bronx,Riverdale,40.890834,-73.912585,0
5,Bronx,Kingsbridge,40.881687,-73.902818,0
6,Manhattan,Marble Hill,40.876551,-73.910660,2
7,Bronx,Woodlawn,40.898273,-73.867315,0
8,Bronx,Norwood,40.877224,-73.879391,0
9,Bronx,Williamsbridge,40.881039,-73.857446,0


In [165]:
# Using geopy library to retrieve the latitude and longitude values of New York City
address = 'New York City, NY'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
ny_latitude = location.latitude
ny_longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(ny_latitude, ny_longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [166]:
newyork_data.dtypes

Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
Code               int8
dtype: object

In [167]:
# create map of New York using latitude and longitude values with neighbohoods superimposed on top
newyork_map = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=10)

ny_neighs = 5

# set color scheme for the clusters
x = np.arange(ny_neighs)
ys = [i+x+(i*x)**2 for i in range(ny_neighs)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, borough, neighborhood, code in zip(newyork_data['Latitude'], newyork_data['Longitude'], newyork_data['Borough'], newyork_data['Neighborhood'],newyork_data['Code']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[code-1],
        fill=True,
        fill_color=rainbow[code-1],
        fill_opacity=0.7,
        parse_html=False).add_to(newyork_map)  
    
newyork_map

In [168]:
# Segment and cluster only the neighborhoods in Manhattan
manhattan_borough = ny_neighborhoods[ny_neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_borough.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [169]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
man_latitude = location.latitude
man_longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(man_latitude, man_longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [170]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[man_latitude, man_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_borough['Latitude'], manhattan_borough['Longitude'], manhattan_borough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

# Explore Manhattan neighborhood with Foursquare API

In [171]:
#Function to explore neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [172]:
manhattan_venues = getNearbyVenues(names=manhattan_borough['Neighborhood'],
                                   latitudes=manhattan_borough['Latitude'],
                                   longitudes=manhattan_borough['Longitude']
                                  )

Found 3314 venues in 40 neighborhoods.


In [173]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3314, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant


In [174]:
#How many venues are returned for each Neighborhood
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,44,44,44,44,44,44
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,47,47,47,47,47,47
East Village,100,100,100,100,100,100


In [175]:
# Find out how many unique categories from all returned venues
print('There are {} distinct venues in {} categories.'.format(
    len(manhattan_venues['Venue'].unique()),len(manhattan_venues['Venue Category'].unique())))

There are 2883 distinct venues in 330 categories.


In [176]:
#Analyze each Neighborhood

# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,S

In [177]:
#Examine the new dataframe size
manhattan_onehot.shape

(3314, 331)

In [178]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,S

In [179]:
#Examine the new dataframe size
manhattan_grouped.shape

(40, 331)

In [180]:
# Print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0         Coffee Shop  0.08
1                Park  0.07
2               Hotel  0.05
3           Wine Shop  0.03
4  Italian Restaurant  0.03


----Carnegie Hill----
            venue  freq
0     Pizza Place  0.06
1  Cosmetics Shop  0.05
2     Coffee Shop  0.05
3            Café  0.04
4             Bar  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1  Gym / Fitness Center  0.05
2           Art Gallery  0.05
3        Cosmetics Shop  0.05
4     French Restaurant  0.05


----Chelsea----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.06
2      Ice Cream Shop  0.05
3              Bakery  0.04
4           Nightclub  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1        Bubble Tea Shop  0.06
2  Vietnamese Restaurant  0.04
3     Dim Sum Restaurant  0.04
4    American Restaurant  0.04


----Civic Center----
                  

In [181]:
#Put that into a pandas dataframe

#Write a function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [182]:
#Create a new dataframe and display the top ten venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
manhattan_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
manhattan_neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    manhattan_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

manhattan_neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Italian Restaurant,Wine Shop,Cupcake Shop,Food Truck,BBQ Joint,Food Court,Department Store
1,Carnegie Hill,Pizza Place,Cosmetics Shop,Coffee Shop,Café,Spa,Japanese Restaurant,Gym,French Restaurant,Yoga Studio,Bookstore
2,Central Harlem,African Restaurant,Seafood Restaurant,French Restaurant,Fried Chicken Joint,Gym / Fitness Center,Art Gallery,Chinese Restaurant,Cosmetics Shop,American Restaurant,Cafeteria
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,American Restaurant,Nightclub,Bakery,Seafood Restaurant,Theater,Art Gallery,Hotel
4,Chinatown,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Dim Sum Restaurant,Hotpot Restaurant,Salon / Barbershop,Bakery,Noodle House
5,Civic Center,Gym / Fitness Center,Italian Restaurant,Bakery,French Restaurant,Sporting Goods Shop,Cocktail Bar,Coffee Shop,Gym,Park,Sandwich Place
6,Clinton,Theater,Coffee Shop,Italian Restaurant,American Restaurant,Gym / Fitness Center,Wine Shop,Hotel,Spa,Gym,Sandwich Place
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Pizza Place,Spa,Thai Restaurant,Seafood Restaurant,Taco Place,Street Art
8,East Village,Ice Cream Shop,Bar,Wine Bar,Cocktail Bar,Mexican Restaurant,Speakeasy,Pizza Place,Chinese Restaurant,Ramen Restaurant,Café
9,Financial District,Coffee Shop,Hotel,Bar,Gym,Wine Shop,Steakhouse,Italian Restaurant,Food Truck,Pizza Place,Falafel Restaurant


In [183]:
manhattan_neighborhoods_venues_sorted.iloc[11,]

Neighborhood                            Gramercy
1st Most Common Venue         Italian Restaurant
2nd Most Common Venue               Cocktail Bar
3rd Most Common Venue                 Restaurant
4th Most Common Venue                  Wine Shop
5th Most Common Venue                 Bagel Shop
6th Most Common Venue                Coffee Shop
7th Most Common Venue                Pizza Place
8th Most Common Venue         Mexican Restaurant
9th Most Common Venue     Thrift / Vintage Store
10th Most Common Venue             Grocery Store
Name: 11, dtype: object

In [184]:
# Cluster the Neighborhoods using k-means

#Run K-means to cluster neighborhood into 3 clusters

# set number of clusters
kclusters = 3

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 1, 1, 1, 0, 1, 1])

In [185]:
#Create a new dataframe that includes the cluster and the top ten venues for each neighborhood

manhattan_merged = manhattan_borough

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(manhattan_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Coffee Shop,Discount Store,Yoga Studio,Pharmacy,Shoe Store,Tennis Stadium,Seafood Restaurant,Gym,Bank,Sandwich Place
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Dim Sum Restaurant,Hotpot Restaurant,Salon / Barbershop,Bakery,Noodle House
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Bakery,Mobile Phone Shop,Deli / Bodega,Sandwich Place,Pizza Place,Latin American Restaurant,Tapas Restaurant,Park,Grocery Store
3,Manhattan,Inwood,40.867684,-73.921210,1,Café,Mexican Restaurant,Lounge,Pizza Place,Frozen Yogurt Shop,Restaurant,Park,Chinese Restaurant,American Restaurant,Bakery
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Mexican Restaurant,Coffee Shop,Café,Pizza Place,Deli / Bodega,Cocktail Bar,Indian Restaurant,Sandwich Place,Yoga Studio,Chinese Restaurant
5,Manhattan,Manhattanville,40.816934,-73.957385,1,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Food & Drink Shop,Beer Garden,Bike Trail,Sushi Restaurant,Supermarket,Burger Joint
6,Manhattan,Central Harlem,40.815976,-73.943211,1,African Restaurant,Seafood Restaurant,French Restaurant,Fried Chicken Joint,Gym / Fitness Center,Art Gallery,Chinese Restaurant,Cosmetics Shop,American Restaurant,Cafeteria
7,Manhattan,East Harlem,40.792249,-73.944182,0,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Pizza Place,Spa,Thai Restaurant,Seafood Restaurant,Taco Place,Street Art
8,Manhattan,Upper East Side,40.775639,-73.960508,0,Italian Restaurant,Exhibit,Art Gallery,Bakery,Coffee Shop,Juice Bar,Boutique,French Restaurant,Gym / Fitness Center,Hotel
9,Manhattan,Yorkville,40.775930,-73.947118,1,Italian Restaurant,Coffee Shop,Bar,Gym,Pizza Place,Deli / Bodega,Japanese Restaurant,Mexican Restaurant,Sushi Restaurant,Park


In [186]:
# create map
manhattan_map_clusters = folium.Map(location=[man_latitude, man_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(manhattan_map_clusters)
       
manhattan_map_clusters

In [187]:
# Examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

manhattan_cluster_0 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
manhattan_cluster_0

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Dim Sum Restaurant,Hotpot Restaurant,Salon / Barbershop,Bakery,Noodle House
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Deli / Bodega,Sandwich Place,Pizza Place,Latin American Restaurant,Tapas Restaurant,Park,Grocery Store
4,Hamilton Heights,Mexican Restaurant,Coffee Shop,Café,Pizza Place,Deli / Bodega,Cocktail Bar,Indian Restaurant,Sandwich Place,Yoga Studio,Chinese Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Pizza Place,Spa,Thai Restaurant,Seafood Restaurant,Taco Place,Street Art
8,Upper East Side,Italian Restaurant,Exhibit,Art Gallery,Bakery,Coffee Shop,Juice Bar,Boutique,French Restaurant,Gym / Fitness Center,Hotel
13,Lincoln Square,Gym / Fitness Center,Theater,Plaza,Italian Restaurant,Concert Hall,Café,French Restaurant,Indie Movie Theater,Opera House,Performing Arts Venue
15,Midtown,Hotel,Food Truck,Steakhouse,Coffee Shop,Theater,Park,American Restaurant,Sporting Goods Shop,Bakery,Spa
18,Greenwich Village,Italian Restaurant,French Restaurant,Sushi Restaurant,Clothing Store,Chinese Restaurant,Indian Restaurant,Seafood Restaurant,Café,Caribbean Restaurant,Burger Joint
19,East Village,Ice Cream Shop,Bar,Wine Bar,Cocktail Bar,Mexican Restaurant,Speakeasy,Pizza Place,Chinese Restaurant,Ramen Restaurant,Café
20,Lower East Side,Pizza Place,Café,Coffee Shop,Chinese Restaurant,Latin American Restaurant,Park,Cocktail Bar,Shoe Store,Bakery,Ramen Restaurant


In [188]:
manhattan_cluster_1 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
manhattan_cluster_1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Discount Store,Yoga Studio,Pharmacy,Shoe Store,Tennis Stadium,Seafood Restaurant,Gym,Bank,Sandwich Place
3,Inwood,Café,Mexican Restaurant,Lounge,Pizza Place,Frozen Yogurt Shop,Restaurant,Park,Chinese Restaurant,American Restaurant,Bakery
5,Manhattanville,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Food & Drink Shop,Beer Garden,Bike Trail,Sushi Restaurant,Supermarket,Burger Joint
6,Central Harlem,African Restaurant,Seafood Restaurant,French Restaurant,Fried Chicken Joint,Gym / Fitness Center,Art Gallery,Chinese Restaurant,Cosmetics Shop,American Restaurant,Cafeteria
9,Yorkville,Italian Restaurant,Coffee Shop,Bar,Gym,Pizza Place,Deli / Bodega,Japanese Restaurant,Mexican Restaurant,Sushi Restaurant,Park
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Gym / Fitness Center,Pizza Place,Burger Joint,Gym,Sporting Goods Shop,Mexican Restaurant,Café
11,Roosevelt Island,Park,Sandwich Place,Residential Building (Apartment / Condo),Dog Run,Greek Restaurant,Baseball Field,Gym,Liquor Store,School,Scenic Lookout
12,Upper West Side,Italian Restaurant,Bar,Indian Restaurant,Vegetarian / Vegan Restaurant,Burger Joint,Coffee Shop,Bakery,Wine Bar,Mediterranean Restaurant,Sushi Restaurant
14,Clinton,Theater,Coffee Shop,Italian Restaurant,American Restaurant,Gym / Fitness Center,Wine Shop,Hotel,Spa,Gym,Sandwich Place
16,Murray Hill,Hotel,Coffee Shop,Italian Restaurant,Bar,Gym,Japanese Restaurant,French Restaurant,Spa,Sandwich Place,Salon / Barbershop


In [191]:
manhattan_cluster_2 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
manhattan_cluster_2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,Pizza Place,Cosmetics Shop,Coffee Shop,Café,Spa,Japanese Restaurant,Gym,French Restaurant,Yoga Studio,Bookstore
